In [1]:
from time import time

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, LambdaCallback
from tensorflow.python.keras.layers import Input, LSTM, Dense, Activation, Dropout
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.utils import to_categorical
import numpy as np


timestamp = int(time())

## Dados

Carregamos todo o texto (porque dá) e passamos para minúsculas

In [2]:
from keras.utils.data_utils import get_file
paths = [
    'data/casmurro.txt',
#     'data/jquery.txt',
]
text = ''
for path in paths:
    text += open(path).read()
text = text.lower()

Using TensorFlow backend.


Detectamos os caracteres e mapeamos os índices

In [3]:
chars = sorted(set(text))
char_to_int = {c: i for i, c in enumerate(chars)}
char_to_ohe = {c: to_categorical(i, len(chars)) for i, c in enumerate(chars)}

Geramos os exemplos "sequencia -> proxima letra"

In [4]:
length = 40
x, y = [], []
word = text[:length]
for letter in text[length:]:
    x.append([char_to_ohe[w] for w in word])
    y.append(char_to_ohe[letter])
    word = word[1:] + letter
x, y = np.stack(x[::3]), np.stack(y[::3])

In [5]:
x.shape

(125140, 40, 68)

Embaralhando o dataset

In [6]:
i = np.random.permutation(x.shape[0])
x, y = x[i], y[i]

Separando treino e teste

In [7]:
i = int(.9 * len(x))
x_train, x_test = x[:i], x[i:]
y_train, y_test = y[:i], y[i:]

## Modelo

Camada de entrada (compatível com a forma de x)

In [8]:
out = entry = Input(shape=x_train.shape[1:])

Camada de memória

In [9]:
# out = LSTM(128, return_sequences=True)(out)
out = LSTM(128)(out)

Camada de saída com um neurônio para cada caractere e aplicação do softmax para obtermos uma distribuição de probabilidade

In [10]:
out = Dense(y_train.shape[1])(out)
out = Activation('softmax')(out)

Definição do modelo em si

In [11]:
net = Model(entry, out)

Imprimimos a descrição do modelo

In [12]:
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 68)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               100864    
_________________________________________________________________
dense (Dense)                (None, 68)                8772      
_________________________________________________________________
activation (Activation)      (None, 68)                0         
Total params: 109,636
Trainable params: 109,636
Non-trainable params: 0
_________________________________________________________________


## Treinamento

Definição do custo e da otimização

In [13]:
net.compile(
    loss='categorical_crossentropy',
    # optimizer=RMSprop(lr=0.01),
    optimizer=Adam(lr=0.001),
    metrics=['accuracy'])

Treinamento em si

In [14]:
def on_epoch_end(a, b):
    text_gen = '''Era uma vez '''.lower()
    if len(text_gen) < length:
        text_gen = (' ' * length + text_gen)[-length:]
    for _ in range(500):
        word = text_gen[-length:]

        x_gen = [[char_to_ohe[w] for w in word]]
        x_gen = np.stack(x_gen)
        y_gen = net.predict(x_gen, verbose=0)[0]

        char = np.random.choice(chars, p=y_gen)
        # char = chars[np.argmax(y)]

        text_gen += char

    print()
    print(text_gen.strip())
    print()

net.fit(
    x_train, y_train,
    batch_size=1024,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[
        ModelCheckpoint('save/text.{epoch:02d}.h5'),
        TensorBoard(log_dir='logs/text_{}'.format(timestamp)),
        LambdaCallback(on_epoch_end=on_epoch_end)])

Train on 112626 samples, validate on 12514 samples
Epoch 1/50
111616/112626 [============================>.] - ETA: 0s - loss: 3.1485 - acc: 0.1506
era uma vez e fm oss+trã oovass  dq
 aodosds; go arvrs aotrai eiovmuoeneqes i,damci acad
memiu eme eanouq udaáluesqiap,om
h a la-i aue lzan
s ,u  stx dadmpoaoraa a  u ar oor uteas edeem tqvna vca.coid sr,n
brroa:vsulismacp,oahotracin adlt
qs ,ieaasrie nhey,eeanf  bsne eei
, aeangmso  ogop oit-emllsãae a aavevanod é  aesimav
  aáccqar m  d lvtma teãuremomd  lmçstirup
 seestcvmla çsoa mroauu
stxavuzuev ;a s,qt idvnefi uime o,eimú-se  ;crpjaea e  n.ms ,ssl ;ercvi rememao
ese oppoaliiadamásse osm er.ooec  stuu

112626/112626 [==============================] - 24s 214us/step - loss: 3.1470 - acc: 0.1507 - val_loss: 2.9700 - val_acc: 0.1646
Epoch 2/50
111616/112626 [============================>.] - ETA: 0s - loss: 2.8668 - acc: 0.2065
era uma vez ruesg
ratmi.   ze,a. do args, e ielm
bvinde dorid a-si smccd 
u
-eesvaatesc. ue ,e e ginnnprtaio nuo

Epoch 12/50
111616/112626 [============================>.] - ETA: 0s - loss: 2.0901 - acc: 0.3670
era uma vez ava va, cave, pão brapude, que os conhavro. que , aãbres ello gonta, vhe danam tora. 
ão preu, sem tiiss.; e amura a peras.
a
bencac vagre. poralino esa que pos eunoja caras, me ar gumar ou axalisse meita
palquemes nante, tarcom viro menho; se gradis vrem empo?,

fada, não da veis mee fura a; tecar ciraidenha camesmodindo; duita
nusme tego aradas
cuquende, a quella de perarimetar pora mavi..
a-leido. no para venção nos cossandara ; chusta pimar-miashe a meiicou a seu tonteiss am dalitilpe
feccad

112626/112626 [==============================] - 23s 206us/step - loss: 2.0899 - acc: 0.3669 - val_loss: 2.0954 - val_acc: 0.3673
Epoch 13/50
111616/112626 [============================>.] - ETA: 0s - loss: 2.0698 - acc: 0.3711
era uma vez de 5ro folcas frial-a e lerigando ea ju
pora da dezis osem jestoa o cegel-me que ta comur gos; mos encem cia emiames oungor: i-mo a visço, não posta

111616/112626 [============================>.] - ETA: 0s - loss: 1.9174 - acc: 0.4152
era uma vez didoco que atra o reclebro, os ge ou io ve temo nivanha
procenhadepado, a beitia csiavada. sa ragerasse, travato acreria, perter ou que muvim
ás jásdi cabitudos? como tada-me é que le dima devar antalgem, a-me e dá mala pada naca seu pégra, cais
dacença oopois. a
palana; e a milhomas vazes, vronaer a filharas; mas cemia seniripar ecteção. nos todo de creçações, ne capas a acfiz-lhe quenas terca.»

--opódez sem um gois o lher
anteico.
e. comemor, em que homavia a fia actava horcia, e tumtiria.

-

112626/112626 [==============================] - 23s 208us/step - loss: 1.9174 - acc: 0.4152 - val_loss: 1.9453 - val_acc: 0.4119
Epoch 24/50
111616/112626 [============================>.] - ETA: 0s - loss: 1.9030 - acc: 0.4198- ETA: 2s - los
era uma vez de capate, sana, são de contor lhes ah prinho segria de istermova mos o distellave, tossa vem cambata, o sermes derar.


--meu umpesque?

--vea s

111616/112626 [============================>.] - ETA: 0s - loss: 1.7901 - acc: 0.4522
era uma vez a seir. aportamenhe que tinha e duis aftereo, octermonte
gou, a naviva; não paigo; paquei póde capitú comi, a
olhes, e quental ambenhom, noverquellaa maram taes outro fileal-lhe. pophe mura o falado de semuada ado outra paima não isso, upaquefles. mes de interragor-a,
nos namintodo, e telestam daqueiço esje faitas tudo nocidar, mapai em cheido. octem-nos um matra?» im tommo.... nos passou alto. o montão, de indives catava, que vansor que porque não é parunro, tas om dinto! dispromim a coda, de
d

112626/112626 [==============================] - 24s 211us/step - loss: 1.7900 - acc: 0.4524 - val_loss: 1.8457 - val_acc: 0.4428
Epoch 35/50
111616/112626 [============================>.] - ETA: 0s - loss: 1.7798 - acc: 0.4557
era uma vez de dizer a casa,, coguravaquer á apé pe da, ia tenticinho.
ose senhoros, que jástoria conteu...

não sebarvi.-se, mor ade um
alcum o flòr ao, vidas, mas volhasd

111616/112626 [============================>.] - ETA: 0s - loss: 1.6877 - acc: 0.4821
era uma vez bá de do sem padre, camirao os he emplidamente por mer deixa metraz;
e cár que fia a dinha mair um que o prejonnina
inejados, desma com
mais, por llivia: «poscudide, som hove algunas bosmávra, nos
passantou-me ponso cunscirou que eu ser lhe não, depois de olollo, e como a dracevu-am filho:

--pou despida? agrejeu salia, nova sebra?
--paqée, querjo, vassam fuz,-s e tal as marava-se que obri na rosa.
bendia-me, escalar, conteci de mim puda noturia. toda que a vocou e tejá?

--não as despeito! quad

112626/112626 [==============================] - 24s 210us/step - loss: 1.6879 - acc: 0.4820 - val_loss: 1.7823 - val_acc: 0.4584
Epoch 46/50
111616/112626 [============================>.] - ETA: 0s - loss: 1.6780 - acc: 0.4863
era uma vez adavalde. focie quando dissimmento, estavacáel-o que a vezes ansalmentranto a congeste
umpenteces, dovalidada
explisicio a estandade se
dizendo-le tistou que mi

## Geração de texto

Ponto de partida

In [15]:
text_gen = '''Era uma vez'''.lower()
# text = '''( function( global, factory ) { '''.lower()

Geração em si

In [16]:
if len(text_gen) < length:
    text_gen = (' ' * length + text_gen)[-length:]
for _ in range(100):
    word = text_gen[-length:]

    x_gen = [[char_to_ohe[w] for w in word]]
    x_gen = np.stack(x_gen)
    y_gen = net.predict(x_gen, verbose=0)[0]

    char = np.random.choice(chars, p=y_gen)
    # char = chars[np.argmax(y)]

    text_gen += char

print(text_gen.strip())

era uma vezega, fouto ho do mustinho, peroumas
continte, se
meria-me o mãe, projejudo e um
peggengintes vi ze a
